In [5]:
import numpy as np
import pandas as pd
import os
import random
from shutil import copyfile
#import pydicom as dicom
import cv2

# Paths Definition

In [3]:
BASE_PATH = os.path.dirname(os.path.realpath(__name__))
BASE_PATH

'/Users/alvaromoure/Desarrollo/UPM/TFM/Alvaro-TFM'

### Paths in Ubuntu

In [9]:
SIMM_COVID_DETECTION_PATH = "/media/usuario/WIN10/COVID19/images/SIIM-Detection/siim-covid19-detection"
PAPER_IMAGES_PATH = "/media/usuario/WIN10/COVID19/images/PaperOriginal/"
TEST_PATH = "/media/usuario/WIN10/COVID19/images/test"

### Paths in MAC

In [15]:

PAPER_IMAGES_PATH = "/Volumes/Alvaro HD/UPM/Corpus TFM/COVID/CroppedSegmentedImages/"


# Reading from paper files

In [ ]:
train = []
test = []
test_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
train_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
filename_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
path_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
mapping = {0:'normal',1:'COVID-19',2:'pneumonia'}

In [83]:
for j,file in enumerate(os.listdir(PAPER_IMAGES_PATH)):
    try:
        cat = int(file.split('__')[1])
    except Exception as e:
        cat = int(file.split('_')[1])
    finally:
        count[mapping[cat]] += 1
        entry = [int(count[mapping[cat]]),file,mapping[cat]]
        filename_label[mapping[cat]].append(entry)
print(count)

{'normal': 118383, 'pneumonia': 24101, 'COVID-19': 8010}


In [58]:
filename_label

{'normal': [[1, 'ACT__0__n__PA__CR__ANON1__1__ANON1.png', 'normal'],
  [2, 'ACT__0__n__PA__CR__ANON22__1__ANON22.png', 'normal'],
  [3, 'ACT__0__n__PA__CR__ANON40__1__ANON40.png', 'normal'],
  [4, 'ACT__0__n__PA__CR__ANON56__1__ANON56.png', 'normal']],
 'pneumonia': [],
 'COVID-19': []}

In [84]:
np.random.seed(seed=2)
test_per = 0.1
for key in filename_label.keys():
    if key == 'pneumonia':
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    elif key == 'COVID-19':
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    else:
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    for i,patient in enumerate(filename_label[key]):
        if int(patient[0]) in test_patients:
            test.append(patient)
            test_count[key] += 1
        else:
            train.append(patient)
            train_count[key] += 1

print('test count: ', test_count)
print('train count: ', train_count)

test count:  {'normal': 11838, 'pneumonia': 2410, 'COVID-19': 801}
train count:  {'normal': 106545, 'pneumonia': 21691, 'COVID-19': 7209}


In [85]:
train_file = open(os.path.join(BASE_PATH,"train_split_alvaro.txt"),"a")
for sample in train:
    info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    train_file.write(info)

train_file.close()

test_file = open(os.path.join(BASE_PATH,"test_split_alvaro.txt"), "a")
for sample in test:
    info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    test_file.write(info)

test_file.close()

# Building a Smaller DB for Testing

In [16]:
MAX_IMAGES_PER_CLASS = 8000

In [30]:
train = []
test = []
test_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
train_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
filename_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
path_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
mapping = {0:'normal',1:'COVID-19',2:'pneumonia'}

### For MAC

In [31]:
for j,directory in enumerate(os.listdir(PAPER_IMAGES_PATH)):
    if not directory.startswith('.'):
        for i,file in enumerate(os.listdir(os.path.join(PAPER_IMAGES_PATH,directory))):
            try:
                cat = int(file.split('__')[1])
            except Exception as e:
                cat = int(file.split('_')[1])
            finally:
                if(count[mapping[cat]]<MAX_IMAGES_PER_CLASS):
                    count[mapping[cat]] += 1
                    entry = [int(count[mapping[cat]]),directory + '/' + file,mapping[cat]]
                    filename_label[mapping[cat]].append(entry)
print(count)

{'normal': 8000, 'pneumonia': 8000, 'COVID-19': 8000}


In [32]:
np.random.seed(seed=2)
test_per = 0.1
for key in filename_label.keys():
    if key == 'pneumonia':
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    elif key == 'COVID-19':
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    else:
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    for i,patient in enumerate(filename_label[key]):
        if int(patient[0]) in test_patients:
            test.append(patient)
            test_count[key] += 1
        else:
            train.append(patient)
            train_count[key] += 1

print('test count: ', test_count)
print('train count: ', train_count)

test count:  {'normal': 800, 'pneumonia': 800, 'COVID-19': 800}
train count:  {'normal': 7200, 'pneumonia': 7200, 'COVID-19': 7200}


In [33]:
train_file = open(os.path.join(BASE_PATH,"train_split_small.txt"),"a")
for sample in train:
    info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    train_file.write(info)

train_file.close()

test_file = open(os.path.join(BASE_PATH,"test_split_small.txt"), "a")
for sample in test:
    info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    test_file.write(info)

test_file.close()